In [1]:
import os
import sys
sys.path.append(os.path.abspath('.'))
sys.path.append(os.path.abspath(r'./.'))

In [2]:
import torch


/home/borela/miniconda3/envs/exqa-complearning/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
torch.cuda.empty_cache()

In [4]:
%pwd

'/home/borela/fontes/exqa-complearning/source/calculation'

In [5]:
%cd ../..

/home/borela/fontes/exqa-complearning


In [6]:
# from source.calculation.context_learning.prompt_format import dict_prompt_format    

In [7]:
from source.calculation.context_learning import trata_reader_pipe

In [8]:
texto_pergunta = 'Qual o melhor time do Brasil?'
texto_contexto = 'Por causa da chuva, o Flamengo, melhor time do Brasil, ficou sem jogar a final da Libertadores. Há muitos torcedores pelo Flamengo, o melhor time, na minha Família. Além da bicicleta, em 1990, eu tinha a honra de ser flamenguista.  Ontem, quando reli os documentos do tribunal, descobri que em 1990 quando tomei posse, minha declaração de bens só continha uma bicicleta.'

In [9]:
dict_config = {
                # parâmetros complementares
                "learning_method":'context',                
                "num_top_k": 3,
                "num_max_answer_length":200,
                # parâmetros transfer # deixar branco                
                "num_doc_stride": "",
                "num_factor_multiply_top_k": "",
                "if_handle_impossible_answer": "",
                # parâmetros context
                'list_stop_words': ['.', '\n', '\n','!'],
                'cod_prompt_format': 1.,
                'if_do_sample': False,
                "val_length_penalty":0.,                    
                'val_temperature': 0.1,
}

In [10]:
from source.calculation import squad_evaluate_v1_1

In [11]:
from source.data_related import squad_related

In [12]:
squad_dataset =squad_related.load_squad_dataset_1_1('pt')

Using custom data configuration default-61eba6f0d4a86c5d
Reusing dataset json (/home/borela/.cache/huggingface/datasets/json/default-61eba6f0d4a86c5d/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5)
100%|██████████| 1/1 [00:00<00:00, 449.50it/s]


In [28]:
## Avaliação
import importlib
importlib.reload(squad_evaluate_v1_1)


<module 'source.calculation.squad_evaluate_v1_1' from '/home/borela/fontes/exqa-complearning/source/calculation/squad_evaluate_v1_1.py'>

In [14]:
import importlib
importlib.reload(trata_reader_pipe)

<module 'source.calculation.context_learning.trata_reader_pipe' from '/home/borela/fontes/exqa-complearning/source/calculation/context_learning/trata_reader_pipe.py'>

In [15]:
dict_config

{'learning_method': 'context',
 'num_top_k': 3,
 'num_max_answer_length': 200,
 'num_doc_stride': '',
 'num_factor_multiply_top_k': '',
 'if_handle_impossible_answer': '',
 'list_stop_words': ['.', '\n', '\n', '!'],
 'cod_prompt_format': 1.0,
 'if_do_sample': False,
 'val_length_penalty': 0.0,
 'val_temperature': 0.1}

In [16]:
name_model = "EleutherAI/gpt-j-6B"
reader = trata_reader_pipe.Reader(name_model, dict_config)
print(f"reader.info()\n {reader.info}")

reader.info()
 {'name': 'EleutherAI/gpt-j-6B', 'device': 'cuda:0', 'top_k': 3, 'cod_prompt_format': 1, 'if_do_sample': False, 'val_temperature': 0.1, 'list_stop_words': ['.', '\n', '\n', '!'], 'val_length_penalty': 0.0, 'length_penalty': 0.0, 'doc_stride': '', 'factor_multiply_top_k': '', 'handle_impossible_answer': '', 'max_answer_length': 200, 'max_seq_len': 2048}


In [17]:
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [18]:
response

[{'texto_resposta': ' Flamengo\n\n'},
 {'texto_resposta': ' O Flamengo.'},
 {'texto_resposta': ' Flamengo.\n'}]

In [29]:
dict_config_eval = {"num_question_max":2}

In [30]:
resultado = squad_evaluate_v1_1.evaluate_learning_method_one_by_one_dataset(parm_dataset=squad_dataset,
parm_reader=reader,
parm_dict_config_model=dict_config, 
parm_dict_config_eval=dict_config_eval,
parm_if_record=True,
parm_interval_print=10)

/home/borela/miniconda3/envs/exqa-complearning/lib/python3.7/site-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evalating in dataset squad-dev-v1.1-pt model 
{'name': 'EleutherAI/gpt-j-6B', 'device': 'cuda:0', 'top_k': 3, 'cod_prompt_format': 1, 'if_do_sample': False, 'val_temperature': 0.1, 'list_stop_words': ['.', '\n', '\n', '!'], 'val_length_penalty': 0.0, 'length_penalty': 0.0, 'doc_stride': '', 'factor_multiply_top_k': '', 'handle_impossible_answer': '', 'max_answer_length': 200, 'max_seq_len': 2048} 
Evalating in dataset squad-dev-v1.1-pt model 
{'name': 'EleutherAI/gpt-j-6B', 'device': 'cuda:0', 'top_k': 3, 'cod_prompt_format': 1, 'if_do_sample': False, 'val_temperature': 0.1, 'list_stop_words': ['.', '\n', '\n', '!'], 'val_length_penalty': 0.0, 'length_penalty': 0.0, 'doc_stride': '', 'factor_multiply_top_k': '', 'handle_impossible_answer': '', 'max_answer_length': 200, 'max_seq_len': 2048} 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evaluation result: Evaluation properties: {'name_learning_method': 'context', 'ind_language': 'pt', 'name_model': 'EleutherAI/gpt-j-6B', 'name_device': 'cuda:0', 'descr_filter': "{'num_question_max': 2}", 'datetime_execution': '2022-07-17 21:34:14', 'num_top_k': 3, 'num_factor_multiply_top_k': '', 'num_doc_stride': '', 'num_max_answer_length': 200, 'if_handle_impossible_answer': '', 'list_stop_words': "['.', '\\n', '\\n', '!']", 'if_do_sample': False, 'val_length_penalty': 0.0, 'val_temperature': 0.1, 'cod_prompt_format': 1.0, 'num_question': 2, 'time_execution_total': 3, 'time_execution_per_question': 1539}
Evaluation metrics: ['EM:0.0', 'F1:23.53', 'EM@3:0.0', 'F1@3:24.26'])


In [24]:
resultado = squad_evaluate_v1_1.evaluate_learning_method_one_by_one_dataset(parm_dataset=squad_dataset,
parm_reader=reader,
parm_dict_config_model=dict_config, 
parm_dict_config_eval=dict_config_eval,
parm_if_record=False,
parm_interval_print=10)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evalating in dataset squad-dev-v1.1-pt model 
{'name': 'EleutherAI/gpt-j-6B', 'device': 'cuda:0', 'top_k': 3, 'cod_prompt_format': 1, 'if_do_sample': False, 'val_temperature': 0.1, 'list_stop_words': ['.', '\n', '\n', '!'], 'val_length_penalty': 0, 'length_penalty': 0, 'doc_stride': 0, 'factor_multiply_top_k': 0, 'handle_impossible_answer': False, 'max_answer_length': 200, 'max_seq_len': 2048} 
Evalating in dataset squad-dev-v1.1-pt model 
{'name': 'EleutherAI/gpt-j-6B', 'device': 'cuda:0', 'top_k': 3, 'cod_prompt_format': 1, 'if_do_sample': False, 'val_temperature': 0.1, 'list_stop_words': ['.', '\n', '\n', '!'], 'val_length_penalty': 0, 'length_penalty': 0, 'doc_stride': 0, 'factor_multiply_top_k': 0, 'handle_impossible_answer': False, 'max_answer_length': 200, 'max_seq_len': 2048} 


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Em normalize_answer. Antes:  O time da NFL que representou a AFC no Super Bowl 50 foi o Denver Broncos.; Depois: o time da nfl que representou afc no super bowl 50 foi o denver broncos
Em normalize_answer. Antes: Denver Broncos; Depois: denver broncos
Em normalize_answer. Antes:  O time da NFL que representou a AFC no Super Bowl 50 foi o Denver Broncos.; Depois: o time da nfl que representou afc no super bowl 50 foi o denver broncos
Em normalize_answer. Antes: Denver Broncos; Depois: denver broncos
Em normalize_answer. Antes:  O time da NFL que representou a AFC no Super Bowl 50 foi o Denver Broncos.; Depois: o time da nfl que representou afc no super bowl 50 foi o denver broncos
Em normalize_answer. Antes: Denver Broncos; Depois: denver broncos
Em normalize_answer. Antes:  O time da NFL que representou a AFC no Super Bowl 50 foi o Denver Broncos.; Depois: o time da nfl que representou afc no super bowl 50 foi o denver broncos
Em normalize_answer. Antes: Denver Broncos; Depois: denver 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Em normalize_answer. Antes:  O time da NFL que representou a NFC no Super Bowl 50 foi o Carolina Panthers.; Depois: o time da nfl que representou nfc no super bowl 50 foi o carolina panthers
Em normalize_answer. Antes: Carolina Panthers; Depois: carolina panthers
Em normalize_answer. Antes:  O time da NFL que representou a NFC no Super Bowl 50 foi o Carolina Panthers.; Depois: o time da nfl que representou nfc no super bowl 50 foi o carolina panthers
Em normalize_answer. Antes: Carolina Panthers; Depois: carolina panthers
Em normalize_answer. Antes:  O time da NFL que representou a NFC no Super Bowl 50 foi o Carolina Panthers.; Depois: o time da nfl que representou nfc no super bowl 50 foi o carolina panthers
Em normalize_answer. Antes: Carolina Panthers; Depois: carolina panthers
Em normalize_answer. Antes:  O time da NFL que representou a NFC no Super Bowl 50 foi o Carolina Panthers.; Depois: o time da nfl que representou nfc no super bowl 50 foi o carolina panthers
Em normalize_answ

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Em normalize_answer. Antes:  O Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da Liga Nacional de Futebol Americano (NFL) para a temporada de 2015.; Depois: o super bowl 50 foi um jogo de futebol americano para determinar o campeão da liga nacional de futebol americano nfl para temporada de 2015
Em normalize_answer. Antes: Santa Clara, Califórnia; Depois: santa clara califórnia
Em normalize_answer. Antes:  O Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da Liga Nacional de Futebol Americano (NFL) para a temporada de 2015.; Depois: o super bowl 50 foi um jogo de futebol americano para determinar o campeão da liga nacional de futebol americano nfl para temporada de 2015
Em normalize_answer. Antes: Levi's Stadium na área da Baía de São Francisco em Santa Clara, Califórnia; Depois: levis stadium na área da baía de são francisco em santa clara califórnia
Em normalize_answer. Antes:  O Super Bowl 50 foi um jogo de futebol americano para de

AssertionError: Estrutura esperada de parm_evaluation_qa_data não corresponde ao esperado Key Esperado[cod_format_prompt] not in Encontrado
Key Esperado[list_stop_word] not in Encontrado
Key Encontrado[list_stop_words] not in Esperado
Key Encontrado[cod_prompt_format] not in Esperado


In [20]:
squad_dataset.dataset[0]

{'id': '56be4db0acb8001400a502ec',
 'title': 'Super_Bowl_50',
 'context': 'O Super Bowl 50 foi um jogo de futebol americano para determinar o campeão da Liga Nacional de Futebol Americano (NFL) para a temporada de 2015. O campeão da Conferência Americana de Futebol Americano (AFC), Denver Broncos, derrotou o campeão da Conferência Nacional de Futebol Americano (NFC), Carolina Panthers 24–10, para ganhar seu terceiro título do Super Bowl. O jogo foi disputado em 7 de fevereiro de 2016, no Levi\'s Stadium na área da Baía de São Francisco em Santa Clara, Califórnia. Como este foi o 50º Super Bowl, a liga enfatizou o "aniversário de ouro" com várias iniciativas com temas de ouro, além de suspender temporariamente a tradição de nomear cada jogo do Super Bowl com números romanos (sob os quais o jogo seria conhecido como " Super Bowl L "), de modo que o logotipo possa exibir com destaque os algarismos arábicos 50.',
 'question': 'Qual time da NFL representou a AFC no Super Bowl 50?',
 'answer

In [1]:
response = reader.answer_one_question(texto_pergunta, texto_contexto)

NameError: name 'reader' is not defined

In [11]:
dict_config['num_top_k'] = 1
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' O Flamengo.'}]


In [13]:
dict_config['num_top_k'] = 4
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

chars(prompt): 494
tokens(prompt): 184


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


texto gerado: [{'generated_text': ' Flamengo\n'}, {'generated_text': ' Flamengo\n'}, {'generated_text': ' Flamengo\n'}, {'generated_text': ' Flamengo\n'}]


In [16]:
dict_config['val_temperature'] = 2.
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' O Flamengo.'}, {'generated_text': ' Não dá para'}, {'generated_text': ' O Corinthians, um dos'}, {'generated_text': ' Flamengo.\n'}]


In [9]:
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' Flamengo.'}]


In [10]:
dict_config['num_top_k'] = 3
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' Quem sabe se algum dia o Flamengo vai ao mundial porque quando eles fizeram, a gente ficou sem jogar...\n'}, {'generated_text': ' O Flamengo\n\nA:\n\nAcho que a pergunta pode ser respondida diretamente pelo texto que está usando.  \n\nText'}, {'generated_text': '\n\nTexto: A bicicleta e a honra de ser flamenguista, de um Flamengo.\n\nPossível dificuldade:\nQuase'}]


In [14]:
dict_config['val_temperature'] = 0.1
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' Flamengo\n'}, {'generated_text': ' Flamengo\n'}, {'generated_text': ' Flamengo\n'}]


In [15]:
dict_config['num_top_k'] = 3
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' Flamengo\n'}, {'generated_text': ' Flamengo\n'}, {'generated_text': ' O Flamengo'}]


In [12]:
dict_config['val_temperature'] = 0.5
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' Flamengo\n'}, {'generated_text': ' O Flamengo'}, {'generated_text': ' O Flamengo'}]


In [13]:
dict_config['val_temperature'] = 1.5
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' Flamengo, onnês...\n'}, {'generated_text': ' Flamengo\n\n'}, {'generated_text': ' Palmeiras\nExplico:\nEm'}]


In [ ]:
dict_config['if_do_sample'] = False
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

In [16]:
dict_config['num_top_k'] = 1
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' O Flamengo.'}]


In [19]:
dict_config['num_top_k'] = 4
dict_config['num_max_answer_length'] = 50
dict_config['if_do_sample'] = True
reader.atualiza_parametros_resposta(dict_config)
response = reader.answer_one_question(texto_pergunta, texto_contexto)

/home/borela/miniconda3/envs/exqa-complearning/lib/python3.7/site-packages/transformers/pipelines/base.py:1015: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  UserWarning,
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


chars(prompt): 494
tokens(prompt): 184
texto gerado: [{'generated_text': ' O Flamengo.'}, {'generated_text': ' O Flamengo.'}, {'generated_text': ' O Flamengo.'}, {'generated_text': ' Flamengo\n\n'}]


In [ ]:
import importlib
importlib.reload(trata_reader2)

EleutherAI/gpt-j-6B

In [ ]:
teste_trata_reader2.teste_context_reader(name_model = "EleutherAI/gpt-j-6B")

gpt-neo-2.7B

len(prompt): 487

pipe (trata_reader2)
carga do modelo : 27s
gpu: 12575mb



answer_max_length: 100
    Tempo: menos de 1s
    texto gerado 1 letra

answer_max_length: 200
    texto gerado 5 letras

answer_max_length: 2048

Geração de texto:
    Tempo: 100 s
    Gpu: chegou a 23gb

    Resposta enormeeeee


In [ ]:
teste_trata_reader2.teste_context_reader(name_model = 'EleutherAI/gpt-neo-2.7B')

gpt-neo-1.3b

In [ ]:
teste_trata_reader2.teste_context_reader()


In [ ]:
squad_related.avaliar_dataset_squad_1_1(squad_dataset_pt)

In [ ]:
squad_related.avaliar_dataset_squad_1_1(squad_dataset_en)

## Avaliação

In [ ]:
import importlib
importlib.reload(squad_evaluate_v1_1)

In [ ]:
dict_config = {"doc_stride":128,\
               "top_k":3, \
               "max_answer_length":30, \
               "handle_impossible_answer":False}

In [ ]:
resultado_en = squad_evaluate_v1_1.evaluate_transfer(parm_dataset=squad_dataset_en, dict_config=dict_config)
print(resultado_en)

In [ ]:
raise Exception(f"Cells bellow can not be executed. It is just for history reasons.")

## Outas execuções

In [ ]:
resultado_pt = squad_evaluate_v1_1.evaluate_transfer(squad_dataset_pt)
print(resultado_pt)

In [ ]:
resultado_en = squad_evaluate_v1_1.evaluate_transfer(squad_dataset_en)
print(resultado_en)

 self.doc_stride = 30
 
 tamanho_max_resposta = 40
 
 self.handle_impossible_answer = False

In [ ]:
resultado_en = squad_evaluate_v1_1.evaluate_transfer(squad_dataset_en)
print(resultado_en)

In [ ]:
resultado_pt = squad_evaluate_v1_1.evaluate_transfer(squad_dataset_pt)
print(resultado_pt)